# Проект: Инструментальные методы цифровой экономики
---
# Сергей Шмидт

# План
1. Формулировка задачи
2. Определение шкал и предварительная обработка данных
3. Множество Парето оптимальных альтернатив
4. Нормировка шкал
5. Решение задачи выбора различными способами: метод свертки критериев, метод последовательных уступок, пороговое агрегирование, метод целевого программирования, метод анализа иерархий
6. Выводы


# Введение (формулировка задачи)
В качестве задачи, которую нужно будет решить ЛПР, была выбрана задача подбора ноутбука с учетом его характеристик и предпочтений ЛПР. Пусть есть Студент с бюджетом в $100000$ рублей. Он хочет приобрести ноутбук стоимостью от $80000$ до $100000$ рублей для учебы. При выборе он учитывает следующие параметры ноутбука: цена, разрешение экрана, мощность процессора, объем оперативной памяти (RAM), суммарный объем твердотельных накопителей (SSD), вес ноутбука и операционную систему, с которой он поставляется. Чтобы выбрать ноутбук, он зашел на сайт магазина техники [DNS](https://www.dns-shop.ru/catalog/17a892f816404e77/noutbuki/?q=%D0%BD%D0%BE%D1%83%D1%82%D0%B1%D1%83%D0%BA%D0%B8+%D0%B4%D0%BB%D1%8F+%D1%83%D1%87%D0%B5%D0%B1%D1%8B&order=6&stock=now-today-tomorrow-later-out_of_stock) и отфильтровал их по стоимости от $80000$ до $100000$ рублей. Поскольку Студент не слишком разбирается в компьютерных комплектующих, он решил в качестве процессора считать не конкретную модель (по типу Ryzen 7 7700X), а некую оценку производительности (например, 23000 у. е.), основанную на специальных тестах. Подобные оценки Студент нашел на сайте [Benchmark.Best](https://benchmark.best/en/cpu_table.html). Собрав данные в файлы и импортировав их в форматы данных Python, он приступил к принятию решения о выборе ноутбука

# Определение шкал и предварительная обработка данных

Для сравнения ноутбуков были отобраны 7 критериев, среди которых есть те, которые образуют номинальную шкалу (операционная система), порядковую шкалу (разрешение экрана, процессор, память на диске, оперативная память), шкалу отношений (цена, вес). При этом шкалы веса и цены являются отрицательными, а между значениями признака “Операционная система” задано бинарное отношение (Windows 11 Home, Windows 11 Home Single Language), исходя из предпочтений Студента. Для упрощения работы с таблицей последним сразу были присвоены числовые значения $-$ 0 или 1. Числовое значение по критерию “Процессор” соответствует оценке производительности, осуществленной независимо от предпочтений конкретного Студента.

In [1]:
# импортируем библиотеки и фурнкции, которые нам понадобятся
import csv, pandas as pd
from copy import deepcopy

# для избежания ошибок, связанных с неправильной выгрузкой данных из файлов или иных источников, заранее скопируем коллекции с данными, с которыми мы будем работать (к тому же, данных не слишком много)
# у нас будет 2 словаря: первый словарь содержит информацию об операционных системах и их переводе в систему (1,0), второй словарь содержит информацию о всех процессорах, а также об оценке производительности этих процессоров
# основным массивом данных служит список, содержащий характеристики каждого из ноутбуков
os = {'Windows 11 Home': 1, 'Windows 11 Home Single Language':0}
proc = {'Intel Core i7-13700H': '16964',
 'Intel Core i7-13620H': '15592',
 'AMD Ryzen 7 8845HS': '17305',
 'Intel Core Ultra 7 155H': '9972',
 'Intel Core i5-13420H': '11582',
 'Intel Core i5-12450H': '10574',
 'Intel Core i5-1335U': '9172',
 'Intel Core Ultra 5 125H': '13943',
 'Intel Core i7-1355U': '9261',
 'Intel Core i7-12650': '14072',
 'Ryzen 5 7530U': '9683'}
data = [{'Название': 'ASUS VivoBook S 16 Flip',
  'Цена': '81000',
  'Разрешение_экрана': 2304000,
  'Процессор': 'Ryzen 5 7530U',
  'RAM': '8',
  'SSD': '256',
  'Вес': '1.9',
  'ОС': 'Windows 11 Home'},
 {'Название': 'Acer Swift GO 14',
  'Цена': '81500',
  'Разрешение_экрана': 5184000,
  'Процессор': 'Intel Core i5-1335U',
  'RAM': '16',
  'SSD': '512',
  'Вес': '1.25',
  'ОС': 'Windows 11 Home Single Language'},
 {'Название': 'Acer Swift GO',
  'Цена': '84900',
  'Разрешение_экрана': 5184000,
  'Процессор': 'Intel Core i5-13420H',
  'RAM': '16',
  'SSD': '1000',
  'Вес': '1.25',
  'ОС': 'Windows 11 Home Single Language'},
 {'Название': 'Huawei MateBook D 16',
  'Цена': '85000',
  'Разрешение_экрана': 2304000,
  'Процессор': 'Intel Core i5-13420H',
  'RAM': '16',
  'SSD': '1024',
  'Вес': '1.68',
  'ОС': 'Windows 11 Home'},
 {'Название': 'MSI Modern 15 H AI',
  'Цена': '100000',
  'Разрешение_экрана': 2073600,
  'Процессор': 'Intel Core Ultra 7 155H',
  'RAM': '16',
  'SSD': '512',
  'Вес': '1.9',
  'ОС': 'Windows 11 Home'},
 {'Название': 'Acer Aspire 5',
  'Цена': '87500',
  'Разрешение_экрана': 2073600,
  'Процессор': 'Intel Core i7-12650',
  'RAM': '16',
  'SSD': '512',
  'Вес': '1.77',
  'ОС': 'Windows 11 Home Single Language'},
 {'Название': 'Acer Swift Go 16',
  'Цена': '88000',
  'Разрешение_экрана': 6400000,
  'Процессор': 'Intel Core i7-13620H',
  'RAM': '16',
  'SSD': '1000',
  'Вес': '1.65',
  'ОС': 'Windows 11 Home'},
 {'Название': 'Acer Aspire Vero',
  'Цена': '89000',
  'Разрешение_экрана': 2073600,
  'Процессор': 'Intel Core i7-1355U',
  'RAM': '16',
  'SSD': '1024',
  'Вес': '1.74',
  'ОС': 'Windows 11 Home'},
 {'Название': 'ASUS Vivobook S 14 Flip',
  'Цена': '89000',
  'Разрешение_экрана': 2304000,
  'Процессор': 'Intel Core i5-1335U',
  'RAM': '16',
  'SSD': '512',
  'Вес': '1.5',
  'ОС': 'Windows 11 Home Single Language'},
 {'Название': 'Infinix InBook Air Pro+',
  'Цена': '90000',
  'Разрешение_экрана': 5184000,
  'Процессор': 'Intel Core i7-1355U',
  'RAM': '16',
  'SSD': '1000',
  'Вес': '1',
  'ОС': 'Windows 11 Home Single Language'},
 {'Название': 'ASUS ExpertBook B1',
  'Цена': '90000',
  'Разрешение_экрана': 2073600,
  'Процессор': 'Intel Core i7-1355U',
  'RAM': '16',
  'SSD': '512',
  'Вес': '1.69',
  'ОС': 'Windows 11 Home'},
 {'Название': 'MSI Prestige 14H',
  'Цена': '95000',
  'Разрешение_экрана': 2304000,
  'Процессор': 'Intel Core i5-12450H',
  'RAM': '16',
  'SSD': '512',
  'Вес': '1.49',
  'ОС': 'Windows 11 Home Single Language'},
 {'Название': 'HUAWEI MateBook 14',
  'Цена': '95000',
  'Разрешение_экрана': 5529600,
  'Процессор': 'Intel Core Ultra 5 125H',
  'RAM': '16',
  'SSD': '512',
  'Вес': '1.31',
  'ОС': 'Windows 11 Home Single Language'},
 {'Название': 'Honor MagicBook X 16 Plus',
  'Цена': '95000',
  'Разрешение_экрана': 2304000,
  'Процессор': 'AMD Ryzen 7 8845HS',
  'RAM': '16',
  'SSD': '512',
  'Вес': '1.75',
  'ОС': 'Windows 11 Home'},
 {'Название': 'ASUS Vivobook S 15 OLED',
  'Цена': '97000',
  'Разрешение_экрана': 4665600,
  'Процессор': 'Intel Core Ultra 5 125H',
  'RAM': '16',
  'SSD': '512',
  'Вес': '1.5',
  'ОС': 'Windows 11 Home Single Language'},
 {'Название': 'MSI Prestige 14 Evo',
  'Цена': '100000',
  'Разрешение_экрана': 2304000,
  'Процессор': 'Intel Core i7-13700H',
  'RAM': '16',
  'SSD': '512',
  'Вес': '1.49',
  'ОС': 'Windows 11 Home'},
 {'Название': 'HUAWEI MateBook D 16',
  'Цена': '100000',
  'Разрешение_экрана': 2304000,
  'Процессор': 'Intel Core i7-13700H',
  'RAM': '16',
  'SSD': '1000',
  'Вес': '1.72',
  'ОС': 'Windows 11 Home'}]


original_data = deepcopy(data) #создаем копию наших данных для дальнейшего использования
# выводим таблицу
df = pd.DataFrame(data=original_data)
df


Название    Цена  Разрешение_экрана  \
0     ASUS VivoBook S 16 Flip   81000            2304000   
1            Acer Swift GO 14   81500            5184000   
2               Acer Swift GO   84900            5184000   
3        Huawei MateBook D 16   85000            2304000   
4          MSI Modern 15 H AI  100000            2073600   
5               Acer Aspire 5   87500            2073600   
6            Acer Swift Go 16   88000            6400000   
7            Acer Aspire Vero   89000            2073600   
8     ASUS Vivobook S 14 Flip   89000            2304000   
9     Infinix InBook Air Pro+   90000            5184000   
10         ASUS ExpertBook B1   90000            2073600   
11           MSI Prestige 14H   95000            2304000   
12         HUAWEI MateBook 14   95000            5529600   
13  Honor MagicBook X 16 Plus   95000            2304000   
14    ASUS Vivobook S 15 OLED   97000            4665600   
15        MSI Prestige 14 Evo  100000            2304000   
16       HUAWEI MateBook D 16  100000            2304000   

                  Процессор RAM   SSD   Вес                               ОС  
0             Ryzen 5 7530U   8   256   1.9                  Windows 11 Home  
1       Intel Core i5-1335U  16   512  1.25  Windows 11 Home Single Language  
2      Intel Core i5-13420H  16  1000  1.25  Windows 11 Home Single Language  
3      Intel Core i5-13420H  16  1024  1.68                  Windows 11 Home  
4   Intel Core Ultra 7 155H  16   512   1.9                  Windows 11 Home  
5       Intel Core i7-12650  16   512  1.77  Windows 11 Home Single Language  
6      Intel Core i7-13620H  16  1000  1.65                  Windows 11 Home  
7       Intel Core i7-1355U  16  1024  1.74                  Windows 11 Home  
8       Intel Core i5-1335U  16   512   1.5  Windows 11 Home Single Language  
9       Intel Core i7-1355U  16  1000     1  Windows 11 Home Single Language  
10      Intel Core i7-1355U  16   512  1.69                  Windows 11 Home  
11     Intel Core i5-12450H  16   512  1.49  Windows 11 Home Single Language  
12  Intel Core Ultra 5 125H  16   512  1.31  Windows 11 Home Single Language  
13       AMD Ryzen 7 8845HS  16   512  1.75                  Windows 11 Home  
14  Intel Core Ultra 5 125H  16   512   1.5  Windows 11 Home Single Language  
15     Intel Core i7-13700H  16   512  1.49                  Windows 11 Home  
16     Intel Core i7-13700H  16  1000  1.72                  Windows 11 Home

In [2]:
# изменяем наши данные, приводя все значения к нужным форматам
print("Полный список ноутбуков:".upper())
for laptop in data:
    laptop["Цена"] = int(laptop["Цена"])
    laptop["RAM"] = int(laptop["RAM"])
    laptop["SSD"] = int(laptop["SSD"])
    laptop["Вес"] = float(laptop["Вес"])
    laptop["Процессор"] = int(proc[laptop["Процессор"]])
    laptop['ОС'] = os[laptop['ОС']]
    print(laptop["Название"])
print()



print("Пример ноутбука с характеристиками:".upper())
for tag, value in data[0].items():
    print(f"{tag}: {value}")


# визуализируем обработанные данные, с которыми мы и будем работать
df = pd.DataFrame(data=data)
df


ПОЛНЫЙ СПИСОК НОУТБУКОВ:
ASUS VivoBook S 16 Flip
Acer Swift GO 14
Acer Swift GO
Huawei MateBook D 16
MSI Modern 15 H AI
Acer Aspire 5
Acer Swift Go 16
Acer Aspire Vero
ASUS Vivobook S 14 Flip
Infinix InBook Air Pro+
ASUS ExpertBook B1
MSI Prestige 14H
HUAWEI MateBook 14
Honor MagicBook X 16 Plus
ASUS Vivobook S 15 OLED
MSI Prestige 14 Evo
HUAWEI MateBook D 16

ПРИМЕР НОУТБУКА С ХАРАКТЕРИСТИКАМИ:
Название: ASUS VivoBook S 16 Flip
Цена: 81000
Разрешение_экрана: 2304000
Процессор: 9683
RAM: 8
SSD: 256
Вес: 1.9
ОС: 1


Название    Цена  Разрешение_экрана  Процессор  RAM  \
0     ASUS VivoBook S 16 Flip   81000            2304000       9683    8   
1            Acer Swift GO 14   81500            5184000       9172   16   
2               Acer Swift GO   84900            5184000      11582   16   
3        Huawei MateBook D 16   85000            2304000      11582   16   
4          MSI Modern 15 H AI  100000            2073600       9972   16   
5               Acer Aspire 5   87500            2073600      14072   16   
6            Acer Swift Go 16   88000            6400000      15592   16   
7            Acer Aspire Vero   89000            2073600       9261   16   
8     ASUS Vivobook S 14 Flip   89000            2304000       9172   16   
9     Infinix InBook Air Pro+   90000            5184000       9261   16   
10         ASUS ExpertBook B1   90000            2073600       9261   16   
11           MSI Prestige 14H   95000            2304000      10574   16   
12         HUAWEI MateBook 14   95000            5529600      13943   16   
13  Honor MagicBook X 16 Plus   95000            2304000      17305   16   
14    ASUS Vivobook S 15 OLED   97000            4665600      13943   16   
15        MSI Prestige 14 Evo  100000            2304000      16964   16   
16       HUAWEI MateBook D 16  100000            2304000      16964   16   

     SSD   Вес  ОС  
0    256  1.90   1  
1    512  1.25   0  
2   1000  1.25   0  
3   1024  1.68   1  
4    512  1.90   1  
5    512  1.77   0  
6   1000  1.65   1  
7   1024  1.74   1  
8    512  1.50   0  
9   1000  1.00   0  
10   512  1.69   1  
11   512  1.49   0  
12   512  1.31   0  
13   512  1.75   1  
14   512  1.50   0  
15   512  1.49   1  
16  1000  1.72   1

# Оптимальность по Парето

Точка (в данном случае альтернатива с соответствующим ей набором оценок по критериям) оптимальна по Парето в том случае, если ни одна другая точка не доминирует ее по Парето, то есть не имеет набор оценок, не уступающих первому ни по одному из критериев и превышающий первый хотя бы по одному критерию. Из определения Парето-оптимальности следует и сам метод поиска таких точек: перебор с помощью счётчика. При этом также следует учесть отрицательные шкалы веса и цены. Для решения задачи определения Парето оптимальносит конкретного ноутбука напишем специальную функцию:

In [10]:
# чтобы определить, есть ли доминируемые по Парето ноутбуки, напишем функцию, принимающую на вход два словаря с информацией о ноутбуках и выводящую значение True, если первый ноутбук доминирует по Парето второй и False в противном случае
def is_pareto_optimal(a: dict, b: dict) -> bool:
    "Функция, определяющая, доминирует ли первый ноутбук второй ноутбук по Парето"
    parametrs = ["Цена", "Разрешение_экрана", "Процессор", "RAM", "SSD", "Вес", "ОС"]
    counter = 0
    for i in parametrs:
        if (
            i == "Цена" or i == "Вес"
        ):  # рассматриваем критерии с отрицательными шкалами как отдельный случай
            if a[i] <= b[i]:
                counter += 1
            elif a[i] < b[i]:
                counter += 2
            return False  # есть критерий, по которому a лучше b
        if (
            i != "Цена" and i != "Вес"
        ):  # рассматриваем критерии с положительными шкалами
            if a[i] >= b[i]:
                counter += 1
            elif a[i] > b[i]:
                counter += 2
            return False  # есть критерий, по которому a лучше b
    if counter > 0:
        return True  # a доминирует b
    return False  # a не доминирует b


# пробегаемся по всевозможным парам ноутбуков дважды и сравниваем их на Парето оптимальность при помощи ранее написанной функции
for i in data:
    for j in data:
          if is_pareto_optimal(i, j):
                print(f"{i['Название']} доминирует по Парето {j['Название']}")
print("Нет ноутбуков, доминируемых по Парето")


Нет ноутбуков, доминируемых по Парето


# Нормировка шкал

Для нормировки положительных шкал (чем больше $-$ тем лучше) мы будем использовать два варианта:
1. $a_i=\frac{a_i-\min(a_k)}{\max(a_k)-\min(a_k)}$
2. $a_i=\frac{a_i}{\max(a_k)}$

Для нормировки отрицательных шкал (чем меньшк $-$ тем лучше) мы будем использовать те же два варианта, вычтенные из единицы:
1. $a_i=1-\frac{a_i-\min(a_k)}{\max(a_k)-\min(a_k)}$
2. $a_i=1-\frac{a_i}{\max(a_k)}$

Принцип нормировки по каждому признаку определялся индивидуально. Было учтено, что коэффициент осцилляции для таких признаков как цена и процессор относительно мал (0,21 и 0,66 соответственно). Нормировка по первому принципу несправедливо отображала бы предпочтения пользователя, поэтому использовалась линейная нормировка. От использования нормировки третьего типа было решено отказаться, поскольку работать с отрицательными числами в ключе предпочтений труднее и менее интуитивно

In [4]:
# Пишем 4 функции, каждая из которых будет выдавать нормировку для конкретного значения в соответствие с двумя способами нормировки: (1) и (2). При этом для каждого из способов будет два варианта шкал: положительная/отрицательная шкала
def first_var(a, min_a, max_a) -> float:
    return (a - min_a) / (max_a - min_a)


def second_var(a, min_a, max_a) -> float:
    return a / (max_a)


def first_var_neg(a, min_a, max_a) -> float:
    return 1 - ((a - min_a) / (max_a - min_a))


def second_var_neg(a, min_a, max_a) -> float:
    return 1 - (a / max_a)


main = deepcopy(data)  # создаем копию основных данных для дальнейшей работы

# находим минимальные и максимальные занчения по каждому из критериев
min_price = min(main, key=lambda x: x["Цена"])["Цена"]
max_price = max(main, key=lambda x: x["Цена"])["Цена"]

min_res = min(main, key=lambda x: x["Разрешение_экрана"])["Разрешение_экрана"]
max_res = max(main, key=lambda x: x["Разрешение_экрана"])["Разрешение_экрана"]

min_proc = min(main, key=lambda x: x["Процессор"])["Процессор"]
max_proc = max(main, key=lambda x: x["Процессор"])["Процессор"]

min_ram = min(main, key=lambda x: x["RAM"])["RAM"]
max_ram = max(main, key=lambda x: x["RAM"])["RAM"]

min_ssd = min(main, key=lambda x: x["SSD"])["SSD"]
max_ssd = max(main, key=lambda x: x["SSD"])["SSD"]

min_weight = min(main, key=lambda x: x["Вес"])["Вес"]
max_weight = max(main, key=lambda x: x["Вес"])["Вес"]

# заменяем значения в таблице в соответствие с нормировкой
for laptop in data:
    laptop["Цена"] = second_var_neg(a=laptop["Цена"], min_a=min_price, max_a=max_price)
    laptop["Разрешение_экрана"] = first_var(
        a=laptop["Разрешение_экрана"], min_a=min_res, max_a=max_res
    )
    laptop["Процессор"] = second_var(
        a=laptop["Процессор"], min_a=min_proc, max_a=max_proc
    )
    laptop["RAM"] = first_var(a=laptop["RAM"], min_a=min_ram, max_a=max_ram)
    laptop["SSD"] = first_var(a=laptop["SSD"], min_a=min_ssd, max_a=max_ssd)
    laptop["Вес"] = first_var_neg(a=laptop["Вес"], min_a=min_weight, max_a=max_weight)

# выводим таблицу
df = pd.DataFrame(data=data)
df


Название   Цена  Разрешение_экрана  Процессор  RAM  \
0     ASUS VivoBook S 16 Flip  0.190           0.053254   0.559549  0.0   
1            Acer Swift GO 14  0.185           0.718935   0.530020  1.0   
2               Acer Swift GO  0.151           0.718935   0.669286  1.0   
3        Huawei MateBook D 16  0.150           0.053254   0.669286  1.0   
4          MSI Modern 15 H AI  0.000           0.000000   0.576250  1.0   
5               Acer Aspire 5  0.125           0.000000   0.813175  1.0   
6            Acer Swift Go 16  0.120           1.000000   0.901011  1.0   
7            Acer Aspire Vero  0.110           0.000000   0.535163  1.0   
8     ASUS Vivobook S 14 Flip  0.110           0.053254   0.530020  1.0   
9     Infinix InBook Air Pro+  0.100           0.718935   0.535163  1.0   
10         ASUS ExpertBook B1  0.100           0.000000   0.535163  1.0   
11           MSI Prestige 14H  0.050           0.053254   0.611037  1.0   
12         HUAWEI MateBook 14  0.050           0.798817   0.805721  1.0   
13  Honor MagicBook X 16 Plus  0.050           0.053254   1.000000  1.0   
14    ASUS Vivobook S 15 OLED  0.030           0.599112   0.805721  1.0   
15        MSI Prestige 14 Evo  0.000           0.053254   0.980295  1.0   
16       HUAWEI MateBook D 16  0.000           0.053254   0.980295  1.0   

         SSD       Вес  ОС  
0   0.000000  0.000000   1  
1   0.333333  0.722222   0  
2   0.968750  0.722222   0  
3   1.000000  0.244444   1  
4   0.333333  0.000000   1  
5   0.333333  0.144444   0  
6   0.968750  0.277778   1  
7   1.000000  0.177778   1  
8   0.333333  0.444444   0  
9   0.968750  1.000000   0  
10  0.333333  0.233333   1  
11  0.333333  0.455556   0  
12  0.333333  0.655556   0  
13  0.333333  0.166667   1  
14  0.333333  0.444444   0  
15  0.333333  0.455556   1  
16  0.968750  0.200000   1

# Метод амплитудных весов

Для данного метода авторами работы предварительно были проранжированны все критерии, принимая во внимание предпочтения студентов, после чего каждому из них был присвоен вес по системе SMART (см. код)

Важность критериев была выбрана следующей:
1. Процессор
2. SSD
3. RAM
4. Цена
5. Вес
6. Разрешение экрана
7. ОС

In [5]:
# создаем словарь smart, в котором будут храниться баллы, которые ЛПР ставит каждому из критериев
smart = {
    "ОС": 10,
    "Разрешение_экрана": 15,
    "Вес": 25,
    "Цена": 30,
    "RAM": 40,
    "SSD": 45,
    "Процессор": 50,
}
# находим веса всех критериев
imp_price = smart["Цена"] / sum(smart.values())
imp_res = smart["Разрешение_экрана"] / sum(smart.values())
imp_proc = smart["Процессор"] / sum(smart.values())
imp_ram = smart["RAM"] / sum(smart.values())
imp_ssd = smart["SSD"] / sum(smart.values())
imp_weight = smart["Вес"] / sum(smart.values())
imp_os = smart["ОС"] / sum(smart.values())

first_method = list() # создаем переменную для вывода результата
# ставим в соответствие каждому ноутбуку сумму его баллов с учетом весов критериев
for laptop in data:
    first_method.append(
        (
            laptop["Название"],
            laptop["Цена"] * imp_price
            + laptop["Разрешение_экрана"] * imp_res
            + laptop["Процессор"] * imp_proc
            + laptop["RAM"] * imp_ram
            + laptop["SSD"] * imp_ssd
            + laptop["Вес"] * imp_weight
            + laptop["ОС"] * imp_os,
        )
    )

print(
    f"Лучшим ноутбуком для Студента будет модель: {max(first_method, key = lambda x: x[1])[0]}"
)


Лучшим ноутбуком для Студента будет модель: Acer Swift Go 16


# Метод последовательных уступок

Теперь необходимо задать величину уступки, при это важно не только сохранить отношения между предпочтениями, но и учесть значения признаков в обработанной таблице, чтобы шаг уступки не был слишком велик. В противном случае критерии цены и процессора не влияли бы на отбор ноутбука.

In [6]:
# задаем максимальные уступки для каждого из критериев
eps_price = 0.1
eps_res = 0.34
eps_proc = 0.15
eps_ram = 0.24
eps_ssd = 0.2
eps_weight = 0.73
eps_os = 0.5

# находим минимальные и максимальные значения для каждого из критериев для таблицы с нормировкой
min_price = min(data, key=lambda x: x["Цена"])["Цена"]
max_price = max(data, key=lambda x: x["Цена"])["Цена"]
min_res = min(data, key=lambda x: x["Разрешение_экрана"])["Разрешение_экрана"]
max_res = max(data, key=lambda x: x["Разрешение_экрана"])["Разрешение_экрана"]
min_proc = min(data, key=lambda x: x["Процессор"])["Процессор"]
max_proc = max(data, key=lambda x: x["Процессор"])["Процессор"]
min_ram = min(data, key=lambda x: x["RAM"])["RAM"]
max_ram = max(data, key=lambda x: x["RAM"])["RAM"]
min_ssd = min(data, key=lambda x: x["SSD"])["SSD"]
max_ssd = max(data, key=lambda x: x["SSD"])["SSD"]
min_weight = min(data, key=lambda x: x["Вес"])["Вес"]
max_weight = max(data, key=lambda x: x["Вес"])["Вес"]

# фильтруем ноутбуки по каждому критерию и выводим те, которые прошли проверку по каждому из критериев (не фильтруем ноутбуки по показателю ОС, так как этим не очень важным критерием можно пренебречь)
print("Наиболее предпочтительный ноутбук: ")
second_method = list(
    filter(
        lambda x: x["Вес"] > max_weight - eps_weight,
        list(
            filter(
                lambda x: x["RAM"] > max_ram - eps_ram,
                list(
                    filter(
                        lambda x: x["Разрешение_экрана"] > max_res - eps_res,
                        list(
                            filter(
                                lambda x: x["Цена"] > max_price - eps_price,
                                list(
                                    filter(
                                        lambda x: x["SSD"] > max_ssd - eps_ssd,
                                        list(
                                            filter(
                                                lambda x: x["Процессор"]
                                                > max_proc - eps_proc,
                                                data,
                                            )
                                        ),
                                    )
                                ),
                            )
                        ),
                    )
                ),
            )
        ),
    )
)
print(second_method[0]['Название'])
print(f" (всего ноутбуков: {len(second_method)})")


Наиболее предпочтительный ноутбук: 
Acer Swift Go 16
 (всего ноутбуков: 1)


# Целевое программирование

Наиболее просто реализуемый с точки зрения программирования отбор осуществляется с помощью минимизации евклидовых расстояний (более универсально, чем Манхэттэнское расстояние или расстояние Чебышева) по каждому признаку от данной точки до целевой (идеальной) точки, то есть набора наибольших возможных числовых значений по каждому признаку.

In [7]:
# определяем функцию для рассчета евклидова расстояния
def dist(a: dict) -> float:
    'Функция для рассчета евклидова расстояния для различных ноутбуков'
    parametrs = ["Цена", "Разрешение_экрана", "Процессор", "RAM", "SSD", "Вес", "ОС"]
    return (
        imp_price * (max_price - a["Цена"])
        + imp_res * (max_res - a["Разрешение_экрана"])
        + imp_proc * (max_proc - a["Процессор"])
        + imp_ram * (max_ram - a["RAM"])
        + imp_ssd * (max_ssd - a["SSD"])
        + imp_weight * (max_weight - a["Вес"])
    ) ** (0.5)

# находим и выводим ноутбук с наименьшим евклидовым расстоянием до идеального ноутбука (со всеми максимальными показателями)
print("Лучший ноутбук:")
print(min(data, key=dist)["Название"])


Лучший ноутбук:
Acer Swift Go 16


# Пороговое агрегирование

Для решения задачи методом порогового агрегирования ЛПР должен задать отображение множества значений для каждого из критериев в шкалу (1-3). Затем происходит две итерации цикла, в каждой из которых из списка всех ноутбуков удаляются ноутбуки, имеющие оценку "1", "2" соответственно хотя бы по одному из критериев.

In [8]:
# создаем словари и функции для перевода значений всех критериев в шкалу (1,2,3)
mark_proc = {
    "Intel Core Ultra 7 155H": 1,
    "Intel Core i5-1335U": 1,
    "Intel Core i7-1355U": 1,
    "Ryzen 5 7530U": 1,
    "Intel Core i5-13420H": 2,
    "Intel Core i5-12450H": 2,
    "Intel Core Ultra 5 125H": 2,
    "Intel Core i7-13700H": 3,
    "Intel Core i7-13620H": 3,
    "AMD Ryzen 7 8845HS": 3,
    "Intel Core i7-12650": 3,
}
mark_ram = {"8": 2, "16": 3}
mark_ssd = {"256": 1, "512": 2, "1000": 3, "1024": 3}
mark_res = {2073600: 1, 2304000: 1, 4665600: 2, 5184000: 2, 5529600: 3, 6400000: 3}
mark_os = {"0": 2, "1": 3}


def mark_price(x: float) -> int:
    if 0 <= x <= 0.1:
        return 1
    elif 0.1 < x <= 0.15:
        return 2
    elif 0.15 < x:
        return 3


def mark_weight(x: float) -> int:
    if 0 <= x <= 0.2:
        return 1
    elif 0.2 < x <= 0.4:
        return 2
    elif 0.4 < x:
        return 3

# создаем словарь, в котором ключом будет название ноутбука, а значением - список оценок по всем критериям для этого ноутбука
laptops = [i["Название"] for i in data]
mark_laptops = {}
for i in laptops:
    for j in original_data:
        if j["Название"] == i:
            mark_laptops[i] = [
                mark_proc[j["Процессор"]],
                mark_ram[j["RAM"]],
                mark_ssd[j["SSD"]],
                mark_res[j["Разрешение_экрана"]],
                mark_price(float(j["Цена"])),
                mark_weight(float(j["Вес"])),
            ]

# первая итерация: отсеиваем те ноутбуки, у которых есть оценка "1"
result_ = list(
    filter(
        lambda x: x[1].count(1) == 0,
        sorted(mark_laptops.items(), key=lambda x: (x[1].count(1), x[1].count(2))),
    )
)
print(
    "Лучшие ноутбуки после первой итерации:",
    ", \n".join([i[0] for i in result_]),
    sep="\n",
)
print()

# вторая итерация: отсеиваем те ноутбуки, у которых есть оценка "2"
result = list(filter(lambda x: x[1].count(2) == 0, result_))
print(
    "Лучшие ноутбуки после второй итерации:",
    ", \n".join([i[0] for i in result]),
    sep="\n",
)
# так как мы переводили оценки в систему (1-3), то третья итерация не потребуется


Лучшие ноутбуки после первой итерации:
Acer Swift Go 16, 
Acer Swift GO, 
HUAWEI MateBook 14, 
ASUS Vivobook S 15 OLED

Лучшие ноутбуки после второй итерации:
Acer Swift Go 16


# Метод анализа иерархий

После отбора $4$ ноутбуков было решено сократить количество критериев для уменьшения количества принимаемых ЛПР решений. Из рассмотрения удален признак "ОС", "Разрешение экрана", "Вес" за относительной малостью их весов. Далее вручную (Во избежание ошибок матрицы дублировались в Excel, где затем проводилась самопроверка) были заполнены матрицы сравнения альтернатив между собой и критериев между собой. Все матрицы (см. ниже) удовлетворяют условиям положительности и обратной симметричности, а также согласованности, в чём авторы убедились, получив уже ожидаемый результат.


In [9]:
print('Ноутбуки, которые мы сравниваем: ')
print(*[i[0] for i in result_], sep=', ') # наши ноутбуки хранятся в переменной result_
# в качестве наиболее важных критериев делаем сравнение по цене, процессору, ssd, весу

# вручную заполняем матрицы для определения значимости исследуемых в данном пункте критериев с учетом соблюдения свойства сверхтранзитивности

matr_price = [
    [1, 1/2, 4, 5],
    [2, 1, 5, 7],
    [1/4, 1/5, 1, 2],
    [1/5, 1/7, 1/2, 1],
]
sum_matr_price = sum([sum(i) for i in matr_price])



matr_proc = [
    [1, 3, 4, 4],
    [1/3, 1, 1/4, 1/4],
    [4, 4, 1, 1],
    [1/4, 4, 1, 1],
]
sum_matr_proc = sum([sum(i) for i in matr_proc])



matr_ssd = [
    [1, 1, 5, 5],
    [1, 1, 5, 5],
    [1/5, 1/5, 1, 1],
    [1/5, 1/5, 1, 1],
]
sum_matr_ssd = sum([sum(i) for i in matr_ssd])



matr_weight = [
    [1, 1/7, 1/6, 1/3],
    [7, 1, 2, 5],
    [6, 1/2, 1, 4],
    [3, 1/5, 1/4, 1],
]
sum_matr_weight = sum([sum(i) for i in matr_weight])



matr_criteria = [
    [1, 2, 1/5, 1/4],
    [1/2, 1, 1/6, 1/5],
    [5, 6, 1, 2],
    [4, 5, 1/2, 1],
]
sum_matr_criteria = sum([sum(i) for i in matr_criteria])


# итоговая матрица, содержащая все веса для каждого критерия и ноутбука
result_matr = [
    [
        sum(matr_criteria[0]) / sum_matr_criteria,
        sum(matr_criteria[1]) / sum_matr_criteria,
        sum(matr_criteria[2]) / sum_matr_criteria,
        sum(matr_criteria[3]) / sum_matr_criteria,
    ],
    [
        sum(matr_price[0]) / sum_matr_price,
        sum(matr_price[1]) / sum_matr_price,
        sum(matr_price[2]) / sum_matr_price,
        sum(matr_price[3]) / sum_matr_price,
    ],
    [
        sum(matr_weight[0]) / sum_matr_weight,
        sum(matr_weight[1]) / sum_matr_weight,
        sum(matr_weight[2]) / sum_matr_weight,
        sum(matr_weight[3]) / sum_matr_weight,
    ],
    [
        sum(matr_proc[0]) / sum_matr_proc,
        sum(matr_proc[1]) / sum_matr_proc,
        sum(matr_proc[2]) / sum_matr_proc,
        sum(matr_proc[3]) / sum_matr_proc,
    ],
    [
        sum(matr_ssd[0]) / sum_matr_ssd,
        sum(matr_ssd[1]) / sum_matr_ssd,
        sum(matr_ssd[2]) / sum_matr_ssd,
        sum(matr_ssd[3]) / sum_matr_ssd,
    ],
]


# рассчитываем и выводим все исследуемые ноутбуки с их оценкой
print()
print(f'Acer Swift Go 16: {result_matr[0][0]*result_matr[1][0]+result_matr[0][1]*result_matr[2][0]+result_matr[0][2]*result_matr[3][0]+result_matr[0][3]*result_matr[4][0]}')
print(f'Acer Swift GO: {result_matr[0][0]*result_matr[1][1]+result_matr[0][1]*result_matr[2][1]+result_matr[0][2]*result_matr[3][1]+result_matr[0][3]*result_matr[4][1]}')
print(f'HUAEWEI MateBook 14: {result_matr[0][0]*result_matr[1][2]+result_matr[0][1]*result_matr[2][2]+result_matr[0][2]*result_matr[3][2]+result_matr[0][3]*result_matr[4][2]}')
print(f'ASUS Vivobook S 15 OLED: {result_matr[0][0]*result_matr[1][3]+result_matr[0][1]*result_matr[2][3]+result_matr[0][2]*result_matr[3][3]+result_matr[0][3]*result_matr[4][3]}')
print()
print('Победитель: Acer Swift Go 16')


Ноутбуки, которые мы сравниваем: 
Acer Swift Go 16, Acer Swift GO, HUAWEI MateBook 14, ASUS Vivobook S 15 OLED

Acer Swift Go 16: 0.3766345511168919
Acer Swift GO: 0.26052053938702424
HUAEWEI MateBook 14: 0.22047753191362218
ASUS Vivobook S 15 OLED: 0.14236737758246173

Победитель: Acer Swift Go 16


# Выводы

Результаты проделанной работы позволяют утверждать, что при неизменности приоритетности критериев от метода к методу и соблюдении статистической строгости, предложенные на рассмотрение методы решения многокритериальных задач комплементарны, то есть показывают одинаковые или схожие результаты. При этом также важно выбрать правильную нормировку для каждого критерия, в противном случае вес одного из них вкупе с абсолютным значением может существенно исказить результат. Слабость такого подхода заключается в недоступности для обывателя, нерациональности человеческих предпочтений (в связи с чем при разных обстоятельствах значения весов, устанавливаемых пользователем даже для одних и тех же признаков, могут отличаться) и сложности для автоматизации из-за необходимости каждый раз изменять важность критериев, которая служит вводной для написания кода, когда новый пользователь приступает к анализу таблицы